<a href="https://colab.research.google.com/github/HIsu1231/AI_INOVATION_SQUARE/blob/master/2conv_1flattern_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from datetime import datetime

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [3]:
i_nodes = mnist.train.images.shape[1]
o_nodes = mnist.train.labels.shape[1]

#hyper_parameter
learning_rate = 0.001
epochs = 30
batch_size = 100

In [4]:
X = tf.placeholder(tf.float32, [None,i_nodes])
T = tf.placeholder(tf.float32, [None,o_nodes])

#28X28X1
A1 = tf.reshape(X, [-1,28,28,1])

In [5]:
#3X3X32 Filter
W2 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
b2 = tf.Variable(tf.random_normal([32]))

#28X28X32
C2 = tf.nn.conv2d(A1, W2, strides=[1,1,1,1], padding='SAME')

Z2 = tf.nn.relu(C2+b2)

#14X14X32
A2 = tf.nn.max_pool(Z2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [6]:
#4X4X64 Filter
W3 = tf.Variable(tf.random_normal([4,4,32,64], stddev=0.01))
b3 = tf.Variable(tf.random_normal([64]))

#14X14X64
C3 = tf.nn.conv2d(A2, W3, strides=[1,1,1,1], padding='SAME')

Z3 = tf.nn.relu(C3+b3)

#7X7X64
A3 = tf.nn.max_pool(Z3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [7]:
A3_flat = tf.reshape(A3, [-1, 7*7*64])

W4 = tf.Variable(tf.random_normal([7*7*64,10], stddev=0.01))
b4 = tf.Variable(tf.random_normal([10]))

Z4 = tf.matmul(A3_flat,W4) + b4

y = tf.nn.softmax(Z4)

In [8]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=T, logits=Z4))

optimizer = tf.train.AdamOptimizer(learning_rate)

train = optimizer.minimize(loss)

In [9]:
predicted_val = tf.argmax(y,1) #예상 값 저장

accuracy_list = tf.cast(tf.equal(tf.argmax(y,1), tf.argmax(T,1)), dtype=tf.float32)  #비교 값 0 or 1로 표현

accuracy = tf.reduce_mean(accuracy_list) #정확도 저장

In [14]:
index_label_prediction_list = []

with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())

    start_time = datetime.now()

    for i in range(epochs):

        total_batch = int(mnist.train.num_examples / batch_size)

        for step in range(total_batch):

            batch_x_data, batch_t_data = mnist.train.next_batch(batch_size)

            loss_val, _ = sess.run([loss, train], feed_dict={X: batch_x_data, T: batch_t_data})

            if step % 100 == 0:

                print("epochs = ", i+1, ", step = ", step, ", loss value = ", loss_val)

    end_time = datetime.now()
    print("\nElapsed time = ", end_time - start_time)

    temp_list = []

    test_x_data = mnist.test.images
    test_t_data = mnist.test.labels

    accuracy_val, index_label, predicted_val_list = sess.run([accuracy, accuracy_list, predicted_val], feed_dict={X: test_x_data, T: test_t_data})

    print("\nAccuracy = ", accuracy_val)

    for i in range(len(index_label)):

        if index_label[i] == 0:
            
            temp_list.append(i)
            temp_list.append(np.argmax(test_t_data[i]))
            temp_list.append(predicted_val_list[i])

            index_label_prediction_list.append(temp_list)

            temp_list = []

epochs =  1 , step =  0 , loss value =  2.4748213
epochs =  1 , step =  100 , loss value =  2.128204
epochs =  1 , step =  200 , loss value =  0.45483327
epochs =  1 , step =  300 , loss value =  0.39113724
epochs =  1 , step =  400 , loss value =  0.34968498
epochs =  1 , step =  500 , loss value =  0.34221882
epochs =  2 , step =  0 , loss value =  0.20606312
epochs =  2 , step =  100 , loss value =  0.22519515
epochs =  2 , step =  200 , loss value =  0.09740227
epochs =  2 , step =  300 , loss value =  0.063484594
epochs =  2 , step =  400 , loss value =  0.26132417
epochs =  2 , step =  500 , loss value =  0.24530259
epochs =  3 , step =  0 , loss value =  0.066670254
epochs =  3 , step =  100 , loss value =  0.06798753
epochs =  3 , step =  200 , loss value =  0.12706469
epochs =  3 , step =  300 , loss value =  0.13254963
epochs =  3 , step =  400 , loss value =  0.070437804
epochs =  3 , step =  500 , loss value =  0.12524416
epochs =  4 , step =  0 , loss value =  0.06919532
e

In [15]:
print(index_label_prediction_list)

[[18, 3, 8], [115, 4, 9], [158, 3, 2], [259, 6, 0], [321, 2, 7], [326, 2, 1], [340, 5, 3], [358, 7, 4], [445, 6, 0], [571, 4, 9], [582, 8, 2], [583, 2, 7], [613, 2, 8], [619, 1, 8], [659, 2, 1], [720, 5, 8], [740, 4, 9], [813, 9, 8], [846, 7, 9], [882, 9, 7], [939, 2, 0], [947, 8, 9], [1014, 6, 0], [1039, 7, 2], [1181, 6, 1], [1182, 6, 5], [1226, 7, 2], [1232, 9, 4], [1242, 4, 9], [1247, 9, 5], [1263, 4, 9], [1272, 5, 4], [1289, 5, 9], [1319, 8, 0], [1349, 2, 3], [1522, 7, 9], [1621, 0, 6], [1709, 9, 5], [1721, 7, 9], [1751, 4, 1], [1790, 2, 7], [1800, 6, 4], [1878, 8, 3], [1901, 9, 4], [1982, 6, 8], [2024, 7, 9], [2053, 4, 9], [2098, 2, 0], [2118, 6, 0], [2130, 4, 9], [2135, 6, 1], [2266, 1, 4], [2291, 5, 3], [2329, 0, 2], [2369, 5, 3], [2387, 9, 1], [2462, 2, 0], [2476, 5, 8], [2488, 2, 4], [2514, 4, 9], [2597, 5, 3], [2630, 4, 9], [2654, 6, 1], [2770, 3, 5], [2771, 4, 9], [2780, 2, 3], [2836, 4, 7], [2877, 4, 7], [2896, 8, 0], [2921, 3, 8], [2927, 3, 2], [2939, 9, 5], [2952, 3, 5], 